In [1]:
# Imports
import re
import string

import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup
import html as ihtml

import requests
import sqlite3

In [2]:
# Extract the job title
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [3]:
# Extract the company name
def extract_company(soup):
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-linl-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

In [4]:
# Extract the job location
def extract_location(soup):
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find(name="span", attrs={"class":"location"}).text)
        except:
            locations.append("None")
    return(locations)

In [5]:
# Extract the salary
def extract_salary(soup):
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span", attrs={"class":"salaryText"}).text.replace("\n",""))
        except:
            salaries.append("None")
    return(salaries)

In [6]:
# Extract the url for the full job listing
def extract_url(soup):
    urls = []
    for div1 in soup.find_all(name="div", attrs={"class":"row"}):
        for div2 in div1.find_all(name="div", attrs={"class":"title"}):
            for a in div2.find_all(name="a", href=True):
                urls.append(a['href'])
    return(urls)

# Extract the full job description from the job listing
def extract_desc(urls):
    descs = []
    for url in urls:
        full_url = "https://www.indeed.com" + url
        detail_page = requests.get(full_url)
        detail_soup = BeautifulSoup(detail_page.text, "html.parser")
        
        for div in detail_soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            descs.append(div.text)
    return(descs)

In [7]:
# Extract the job count
def extract_count(soup):
    for div in soup.find_all(name="div", attrs={"id":"searchCountPages"}):
        temp_str = div.text.replace(" ", "")
        temp_count_str = re.search('of(.*)jobs', temp_str)
        count_str = re.sub('[^0-9]','', temp_count_str.group(1))
        count = int(count_str)
    return(count)

In [8]:
# Extract the job listings
#    max_results is a key setting - it represents the # of results to retrieve
#    with roughly 10 results per page, so 10 equals one page of results while 100
#    would represent 10 pages.  It takes about 1.5 hours to scrape all 80 permutations
#    (4 job titles * 20 cities) with max_results = 100
def extract_list(title_name, city_name, st_name):
    max_results = 10
    columns = ["city", "job_title", "company", "location", "salary", "description"]

    city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name
    page = requests.get(city_url)
    soup = BeautifulSoup(page.text, "html.parser")
    max_counter = extract_count(soup)
    print(max_counter, title_name, city_name)
   
    job_title_list = []
    company_list = []
    location_list = []
    salary_list = []
    desc_list = []

    for start in range(0, max_results, 10):
        city_url = "https://www.indeed.com/jobs?q=" + title_name + \
                   "&l=" + city_name + "%2C+" + st_name + \
                   "&start=" + str(start)
        page = requests.get(city_url)
        soup = BeautifulSoup(page.text, "html.parser")

        job_title_list.extend(extract_job_title(soup))
        company_list.extend(extract_company(soup))
        location_list.extend(extract_location(soup))
        salary_list.extend(extract_salary(soup))
        add_urls = extract_url(soup)
        desc_list.extend(extract_desc(add_urls))

    return job_title_list, company_list, location_list, salary_list, desc_list, max_counter

# Each data item is extracted and stored in a list.  The lists are then combined into a dataframe.
def job_db(title, city, st):   
    j_list, c_list, l_list, s_list, d_list, m_counter = extract_list(title, city, st)     
    temp_df = pd.DataFrame(list(zip(j_list, c_list, l_list, s_list, d_list)), 
                          columns = ['job_title', 'company', 'location', 'salary', 'description'])
    temp_df['counts'] = m_counter
    city_name = city.replace('+', ' ')
    temp_df['city'] = city_name
    return temp_df

In [9]:
# Data scientists in 20 cities
temp1_df = job_db('data+scientist', 'San+Jose', 'CA')
temp2_df = job_db('data+scientist', 'San+Francisco', 'CA')
temp3_df = job_db('data+scientist', 'Seattle', 'WA')
temp4_df = job_db('data+scientist', 'Washington', 'DC')
temp5_df = job_db('data+scientist', 'New+York', 'NY')
temp6_df = job_db('data+scientist', 'Baltimore', 'MD')
temp7_df = job_db('data+scientist', 'Boulder', 'CO')
temp8_df = job_db('data+scientist', 'San+Diego', 'CA')
temp9_df = job_db('data+scientist', 'Denver', 'CO')
temp10_df = job_db('data+scientist', 'Huntsville', 'AL')
temp11_df = job_db('data+scientist', 'Colorado+Springs', 'CO')
temp12_df = job_db('data+scientist', 'Houston', 'TX')
temp13_df = job_db('data+scientist', 'Trenton', 'NJ')
temp14_df = job_db('data+scientist', 'Dallas', 'TX')
temp15_df = job_db('data+scientist', 'Columbus', 'OH')
temp16_df = job_db('data+scientist', 'Austin', 'TX')
temp17_df = job_db('data+scientist', 'Philadelphia', 'PA')
temp18_df = job_db('data+scientist', 'Durham', 'NC')
temp19_df = job_db('data+scientist', 'Raleigh', 'NC')
temp20_df = job_db('data+scientist', 'Atlanta', 'GA')

data_scientist_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                       temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                       temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                       temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)

print(data_scientist_df.shape)

1274 data+scientist San+Jose
912 data+scientist San+Francisco
1062 data+scientist Seattle
1171 data+scientist Washington
830 data+scientist New+York
164 data+scientist Baltimore
153 data+scientist Boulder
149 data+scientist San+Diego
169 data+scientist Denver
25 data+scientist Huntsville
6 data+scientist Colorado+Springs
92 data+scientist Houston
90 data+scientist Trenton
259 data+scientist Dallas
68 data+scientist Columbus
187 data+scientist Austin
223 data+scientist Philadelphia
146 data+scientist Durham
152 data+scientist Raleigh
245 data+scientist Atlanta
(305, 7)


In [10]:
# Web developers in 20 cities
temp1_df = job_db('web+developer', 'San+Jose', 'CA')
temp2_df = job_db('web+developer', 'San+Francisco', 'CA')
temp3_df = job_db('web+developer', 'Seattle', 'WA')
temp4_df = job_db('web+developer', 'Washington', 'DC')
temp5_df = job_db('web+developer', 'New+York', 'NY')
temp6_df = job_db('web+developer', 'Baltimore', 'MD')
temp7_df = job_db('web+developer', 'Boulder', 'CO')
temp8_df = job_db('web+developer', 'San+Diego', 'CA')
temp9_df = job_db('web+developer', 'Denver', 'CO')
temp10_df = job_db('web+developer', 'Huntsville', 'AL')
temp11_df = job_db('web+developer', 'Colorado+Springs', 'CO')
temp12_df = job_db('web+developer', 'Houston', 'TX')
temp13_df = job_db('web+developer', 'Trenton', 'NJ')
temp14_df = job_db('web+developer', 'Dallas', 'TX')
temp15_df = job_db('web+developer', 'Columbus', 'OH')
temp16_df = job_db('web+developer', 'Austin', 'TX')
temp17_df = job_db('web+developer', 'Philadelphia', 'PA')
temp18_df = job_db('web+developer', 'Durham', 'NC')
temp19_df = job_db('web+developer', 'Raleigh', 'NC')

web_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(web_developer_df.shape)

1325 web+developer San+Jose
1403 web+developer San+Francisco
1369 web+developer Seattle
3424 web+developer Washington
1361 web+developer New+York
849 web+developer Baltimore
439 web+developer Boulder
354 web+developer San+Diego
593 web+developer Denver
70 web+developer Huntsville
69 web+developer Colorado+Springs
353 web+developer Houston
231 web+developer Trenton
760 web+developer Dallas
224 web+developer Columbus
709 web+developer Austin
509 web+developer Philadelphia
384 web+developer Durham
365 web+developer Raleigh
(355, 7)


In [11]:
# UX designers in 20 cities
temp1_df = job_db('ux+designer', 'San+Jose', 'CA')
temp2_df = job_db('ux+designer', 'San+Francisco', 'CA')
temp3_df = job_db('ux+designer', 'Seattle', 'WA')
temp4_df = job_db('ux+designer', 'Washington', 'DC')
temp5_df = job_db('ux+designer', 'New+York', 'NY')
temp6_df = job_db('ux+designer', 'Baltimore', 'MD')
temp7_df = job_db('ux+designer', 'Boulder', 'CO')
temp8_df = job_db('ux+designer', 'San+Diego', 'CA')
temp9_df = job_db('ux+designer', 'Denver', 'CO')
temp10_df = job_db('ux+designer', 'Huntsville', 'AL')
temp11_df = job_db('ux+designer', 'Colorado+Springs', 'CO')
temp12_df = job_db('ux+designer', 'Houston', 'TX')
temp13_df = job_db('ux+designer', 'Trenton', 'NJ')
temp14_df = job_db('ux+designer', 'Dallas', 'TX')
temp15_df = job_db('ux+designer', 'Columbus', 'OH')
temp16_df = job_db('ux+designer', 'Austin', 'TX')
temp17_df = job_db('ux+designer', 'Philadelphia', 'PA')
temp18_df = job_db('ux+designer', 'Durham', 'NC')
temp19_df = job_db('ux+designer', 'Raleigh', 'NC')
temp20_df = job_db('ux+designer', 'Atlanta', 'GA')

ux_designer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                            temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                            temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                            temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ux_designer_df.shape)

613 ux+designer San+Jose
869 ux+designer San+Francisco
942 ux+designer Seattle
499 ux+designer Washington
864 ux+designer New+York
90 ux+designer Baltimore
131 ux+designer Boulder
103 ux+designer San+Diego
150 ux+designer Denver
9 ux+designer Huntsville
6 ux+designer Colorado+Springs
84 ux+designer Houston
48 ux+designer Trenton
197 ux+designer Dallas
46 ux+designer Columbus
265 ux+designer Austin
155 ux+designer Philadelphia
78 ux+designer Durham
77 ux+designer Raleigh
252 ux+designer Atlanta
(291, 7)


In [12]:
# iOS developers in 20 cities
temp1_df = job_db('ios+developer', 'San+Jose', 'CA')
temp2_df = job_db('ios+developer', 'San+Francisco', 'CA')
temp3_df = job_db('ios+developer', 'Seattle', 'WA')
temp4_df = job_db('ios+developer', 'Washington', 'DC')
temp5_df = job_db('ios+developer', 'New+York', 'NY')
temp6_df = job_db('ios+developer', 'Baltimore', 'MD')
temp7_df = job_db('ios+developer', 'Boulder', 'CO')
temp8_df = job_db('ios+developer', 'San+Diego', 'CA')
temp9_df = job_db('ios+developer', 'Denver', 'CO')
temp10_df = job_db('ios+developer', 'Huntsville', 'AL')
temp11_df = job_db('ios+developer', 'Colorado+Springs', 'CO')
temp12_df = job_db('ios+developer', 'Houston', 'TX')
temp13_df = job_db('ios+developer', 'Trenton', 'NJ')
temp14_df = job_db('ios+developer', 'Dallas', 'TX')
temp15_df = job_db('ios+developer', 'Columbus', 'OH')
temp16_df = job_db('ios+developer', 'Austin', 'TX')
temp17_df = job_db('ios+developer', 'Philadelphia', 'PA')
temp18_df = job_db('ios+developer', 'Durham', 'NC')
temp19_df = job_db('ios+developer', 'Raleigh', 'NC')
temp20_df = job_db('ios+developer', 'Atlanta', 'GA')

ios_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ios_developer_df.shape)

735 ios+developer San+Jose
465 ios+developer San+Francisco
466 ios+developer Seattle
325 ios+developer Washington
311 ios+developer New+York
76 ios+developer Baltimore
100 ios+developer Boulder
75 ios+developer San+Diego
123 ios+developer Denver
13 ios+developer Huntsville
6 ios+developer Colorado+Springs
47 ios+developer Houston
24 ios+developer Trenton
140 ios+developer Dallas
22 ios+developer Columbus
174 ios+developer Austin
85 ios+developer Philadelphia
42 ios+developer Durham
42 ios+developer Raleigh
130 ios+developer Atlanta
(300, 7)


In [13]:
# Combine the data into one dataframe
data_scientist_df['job'] = 'data scientist'
web_developer_df['job'] = 'web developer'
ux_designer_df['job'] = 'ux designer'
ios_developer_df['job'] = 'ios developer'
indeed_df = pd.concat([data_scientist_df, web_developer_df, ux_designer_df, ios_developer_df], ignore_index=True)
indeed_df = indeed_df.drop_duplicates(keep=False) 
indeed_df = indeed_df.reset_index(drop=True)

In [14]:
# Clean up the low and high salaries
def convert_salary(sal_str, s_flag):
    
    sal_split = re.findall(r'\d+', sal_str.replace(",", ""))
    
    if len(sal_split) == 2:
        low_salary = int(sal_split[0])
        high_salary = int(sal_split[1])
    else:
        low_salary = None
        high_salary = None
    
    if s_flag == 'l':
        salary = low_salary
    else:
        salary = high_salary
    
    return salary

indeed_df['low_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'l'), axis=1)
indeed_df['high_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'h'), axis=1)
indeed_df = indeed_df.drop(columns=['salary'])

indeed_df.head()                                      

,job_title,company,location,description,counts,city,job,low_salary,high_salary
0,Data Scientist / Economist,eBay Inc.,None,"Looking for a company that inspires passion, c...",1274,San Jose,data scientist,NaN,NaN
1,Data Scientist,PayPal,None,This role is in the Global Resolutions and Pro...,1274,San Jose,data scientist,NaN,NaN
2,Data Scientist/Machine Learning Engineer,PayPal,None,What does Success Look Like?\nIn your role as ...,1274,San Jose,data scientist,NaN,NaN
3,Associate Data Scientist,PayPal,None,"Design, develop and implement advanced predict...",1274,San Jose,data scientist,NaN,NaN
4,Software Engineer - Machine Learning,Triplebyte,None,Triplebyte is working with a company that is b...,1274,San Jose,data scientist,NaN,NaN


In [15]:
indeed_df.tail()

,job_title,company,location,description,counts,city,job,low_salary,high_salary
1246,iOS Developer,Gozio,"Atlanta, GA",Gozio is seeking an iOS Developer to help buil...,130,Atlanta,ios developer,NaN,NaN
1247,Software Application Developer,Turner,"Atlanta, GA 30319",Opportunity Overview\nThe WarnerMedia Story\nW...,130,Atlanta,ios developer,NaN,NaN
1248,Jr IOS Developer,Adidev Technologies Inc,"Atlanta, GA",Adidev Technologies Inc (MobileDev Power)\n\nw...,130,Atlanta,ios developer,NaN,NaN
1249,Android and/or iOS App Developer,WIKILANE INC.,None,"SUMMARYWikilane Inc., the creator of Invoice H...",130,Atlanta,ios developer,80000.0,120000.0
1250,iOS Developer,Red Voice Technologies,None,"Position Overview: We, at Red Voice Technologi...",130,Atlanta,ios developer,32.0,45.0


In [16]:
# convert to sqlite3 and csv
conn = sqlite3.connect('techsearch.sqlite3')
curs = conn.cursor()
curs.execute('drop table if exists listings')
indeed_df.to_sql('listings', con=conn)
indeed_df.to_csv('techsearch.csv')